20180634_최준혁_NLP_기말과제

## Imports & Define train/valid/test

In [1]:
# Import modules
import nltk
from nltk.corpus import brown

# Define train/valid/test
train = brown.tagged_sents(categories='news')
validation = brown.tagged_sents(categories='editorial')
test = brown.tagged_sents(categories='reviews')

# 카테고리 별 개수 확인
print('train(news) : ',len(brown.words(categories='news')))
print('validation(editorial) : ',len(brown.words(categories='editorial')))
print('test(reviews) : ',len(brown.words(categories='reviews')))

train(news) :  100554
validation(editorial) :  61604
test(reviews) :  40704


## Performance Comparison

### Default

In [2]:
# 여기 패턴부분을 수정하기
patterns = [('.*ing$', 'VBG'),              # gerunds 
            ('.*ed$', 'VBD'),               # simple past
            ('.*es$', 'VBZ'),               # 3rd singular present
            ('.*ould$', 'MD'),              # modals
            ('.*\'s$', 'NN$'),              # possessive nouns
            ('.*s$', 'NNS'),                # plural nouns
            ('^-?[0-9]+(.[0-9]+)?$', 'CD'), # cardinal numbers
            ('.*', 'NN')]                   # nouns (default)

# 여기 수정 불가
t0 = nltk.RegexpTagger(patterns)
t1 = nltk.UnigramTagger(train, backoff=t0)
t2 = nltk.BigramTagger(train, backoff=t1)
t3 = nltk.TrigramTagger(train, backoff=t2)

# valid, test 성능 출력
print("valid_accuracy",  t3.accuracy(validation))
print("test_accuracy",  t3.accuracy(test))

valid_accuracy 0.8636776832673203
test_accuracy 0.8320066823899371


### ANSWER

In [3]:
# 여기 패턴부분을 수정하면서 valid 성능을 확인해보고 성능 향상이 있으면 패턴변경 사항을 수용하는 방식
patterns = [
            ('.*ing$', 'VBG'),                        # 현재 분사 
            ('.*ed$', 'VBD'),                         # 과거 동사
            ('.*ould$', 'MD'),                        # 동사 조동사
            ('.*\'s$', 'NN$'),                        # 소유격 및 명사 단수
            ('.*s$', 'NNS'),                          # 복수명사
            ('^-?[0-9]+(.[0-9]+)?$', 'CD'),           # 기수 숫자
    
            # 형용사류 
            ('.*(nt|erb|ful|al|ic|[aiou]ble)$', 'JJ'),# 형용사용 어미 : 어미 빈도 분석 기반
            ('.*est$', 'JJT'),                        # 최상급 est
            
            # 부사
            ('.*ly$', 'RB'),                          # 대표적 부사형 어미 ly
            
            # 동사
            ('.*(ize|ate)$', 'VB'),                   # 동사형 어미 : 어미 빈도 분석 기반
            
            # 명사
            ('^[A-Z].*', 'NN-TL'),                    # 명사 default : 대문자로 시작하는 것은 NN-TL
            ('.*', 'NN')
            ]                             # 명사 default : 소문자로 시작하는 것은 NN

# 여기 수정 불가
t0 = nltk.RegexpTagger(patterns)
t1 = nltk.UnigramTagger(train, backoff=t0)
t2 = nltk.BigramTagger(train, backoff=t1)
t3 = nltk.TrigramTagger(train, backoff=t2)

# valid 성능 출력
print("valid_accuracy",  t3.accuracy(validation))

valid_accuracy 0.8758846828128044


In [4]:
# valid 성능을 최대한으로 끌어올린 후 test로 성능 확인
acc = t3.accuracy(test)
print("test_accuracy",  acc)

# 과제 최대 점수 확인용
if acc >= 0.85201 : print('\n목표점수(0.85201 이상)에 도달하였습니다')
else : print('목표점수(0.85201 이상)에 도달하지 못했습니다. 더 성능을 올리세요')

test_accuracy 0.8521766902515723

목표점수(0.85201 이상)에 도달하였습니다


## 해결과정전략에 사용했던 코드들

In [5]:
# 명사가 가장 많으므로 default처리하고 빈도 높은 순서대로 품사별 어미파악을 진행함
print(nltk.FreqDist([tag for word, tag in \
               [tagged for row in validation for tagged in row]]).most_common(20))

[('NN', 7675), ('IN', 6204), ('AT', 5311), ('JJ', 3593), ('.', 2988), ('NNS', 2972), (',', 2741), ('VB', 2129), ('NP', 1884), ('CC', 1835), ('RB', 1809), ('VBN', 1491), ('CS', 1194), ('NN-TL', 1152), ('TO', 952), ('VBG', 881), ('MD', 867), ('PPS', 782), ('PP$', 771), ('BEZ', 749)]


In [6]:
# 품사별 어미파악하는 함수 : train_set에서 진행
# 품사를 입력하면 뒤의 4,3,2개 짜리어미가 무엇이 있는지 빈도순으로 보여준다
def umi_check(pos_tag):
    print('-'*100)
    print(f"[{pos_tag}의 어미 빈도 분석 ]")
    print("> 4개 짜리 어미 : ", end = '')
    print(nltk.FreqDist([word[-4:] for word, tag in [tagged for row in train \
                        for tagged in row] if tag == pos_tag]).most_common(10),"\n")

    print("> 3개 짜리 어미 : ", end = '')
    print(nltk.FreqDist([word[-3:] for word, tag in [tagged for row in train \
                        for tagged in row] if tag == pos_tag]).most_common(10),"\n")

    print("> 2개 짜리 어미 : ", end = '')
    print(nltk.FreqDist([word[-2:] for word, tag in [tagged for row in train \
                        for tagged in row] if tag == pos_tag]).most_common(10),"\n")
    print('-'*100)
    
umi_check("JJ")
umi_check("JJT")
umi_check("RB")
umi_check("VB")

----------------------------------------------------------------------------------------------------
[JJ의 어미 빈도 분석 ]
> 4개 짜리 어미 : [('new', 147), ('onal', 144), ('able', 114), ('tive', 112), ('ical', 98), ('cial', 87), ('ious', 75), ('eral', 67), ('ible', 60), ('such', 51)] 

> 3개 짜리 어미 : [('ble', 179), ('nal', 179), ('ive', 151), ('ial', 149), ('new', 147), ('ent', 146), ('cal', 140), ('ous', 137), ('ral', 117), ('ing', 102)] 

> 2개 짜리 어미 : [('al', 760), ('le', 226), ('ic', 213), ('nt', 188), ('ng', 185), ('te', 157), ('ve', 154), ('ew', 150), ('us', 137), ('an', 133)] 

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
[JJT의 어미 빈도 분석 ]
> 4개 짜리 어미 : [('best', 26), ('gest', 25), ('test', 16), ('hest', 8), ('lest', 6), ('iest', 4), ('west', 4), ('nest', 3), ('orst', 2), ('mest', 1)] 

> 3개 짜리 어미 : [('est', 97), ('rst', 2), ('aid', 1)] 

>

In [7]:
# 이 코드는 특정 단어가 품사가 무엇인지 검색해서 확인해보거나 
# 어떤 어미로 끝나는지 확인해보기 위한 코드
# 아래의 예시는 ly로 끝나는 단어들의 품사가 무엇인지 확인하는 것이다 -> 대부분 부사이다
[(word, tag) for word, tag in [tagged for row in test for tagged in row] if word[-2:] == 'ly']

[('Certainly', 'RB'),
 ('lovely', 'JJ'),
 ('only', 'RB'),
 ('considerably', 'QL'),
 ('July', 'NP'),
 ('July', 'NP'),
 ('July', 'NP'),
 ('July', 'NP'),
 ('fly', 'VB'),
 ('firmly', 'QL'),
 ('fly', 'VB'),
 ('wily', 'JJ'),
 ('blandly', 'QL'),
 ('atrociously', 'QL'),
 ('precisely', 'RB'),
 ('hardly', 'QL'),
 ('really', 'RB'),
 ('really', 'RB'),
 ('probably', 'RB'),
 ('wonderfully', 'QL'),
 ('Piccadilly', 'NP'),
 ('truly', 'RB'),
 ('only', 'RB'),
 ('family', 'NN'),
 ('apparently', 'RB'),
 ('entirely', 'QL'),
 ('truly', 'QL'),
 ('inevitably', 'RB'),
 ('affectingly', 'RB'),
 ('soberly', 'RB'),
 ('sincerely', 'RB'),
 ('studiously', 'QL'),
 ('brightly', 'RB'),
 ('gravely', 'RB'),
 ('considerably', 'RB'),
 ('appropriately', 'QL'),
 ('excellently', 'RB'),
 ('helpfully', 'RB'),
 ('comically', 'RB'),
 ('really', 'RB'),
 ('facetiously', 'RB'),
 ('finely', 'QL'),
 ('magnificently', 'QL'),
 ('family', 'NN'),
 ('necessarily', 'RB'),
 ('considerately', 'RB'),
 ('hardly', 'RB'),
 ('cleverly', 'RB'),
 ('un